LAB4


In [7]:
import numpy as np
import pandas as pd
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [2]:
data = pd.read_csv('WineQT.csv')
data = data.drop(columns='Id')
X = data[data.columns[:-1]]
y = data['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [3]:
#определение количества итераций бутстрэпа и количество деревьев
n_iterations = 100
n_trees = 10

#создание списка для сохранения важности признаков для каждого дерева
importance_scores = []

#итерации бутстрэпа
for _ in range(n_iterations):
    #создание бутстрэп-выборки (с повторениями)
    X_boot, y_boot = resample(X, y, replace=True)

    #обучение нескольких деревьев на бутстрэп-выборке
    trees = [DecisionTreeClassifier().fit(X_boot, y_boot) for _ in range(n_trees)]

    #сохранение важности признаков каждого дерева
    for tree in trees:
        importance_scores.append(tree.feature_importances_)

#преобразование списка с важностью признаков в массив numpy для анализа
importance_scores1 = np.array(importance_scores)

#рассчет средней важности признаков и стандартного отклонения важности
mean_importance = np.mean(importance_scores1, axis=0)
std_importance = np.std(importance_scores1, axis=0)

print("Важность признаков каждого дерева:",importance_scores)
print("Средняя важность признаков:",mean_importance)
print("Стандартное отклонение важности:",std_importance)

Важность признаков каждого дерева: [array([0.07759076, 0.10017819, 0.08496448, 0.0796547 , 0.0551131 ,
       0.03704297, 0.1181051 , 0.08240584, 0.10229325, 0.08596982,
       0.17668179]), array([0.09273571, 0.10395114, 0.08574098, 0.07429687, 0.06535349,
       0.04112448, 0.10459992, 0.0761393 , 0.09193014, 0.09144031,
       0.17268766]), array([0.09256048, 0.10842169, 0.08022397, 0.07182852, 0.04027779,
       0.03929843, 0.1104022 , 0.06623615, 0.10071592, 0.10766917,
       0.18236568]), array([0.07876133, 0.11327199, 0.07838187, 0.06874687, 0.06995616,
       0.04335591, 0.11929221, 0.04473031, 0.10673369, 0.09528075,
       0.18148891]), array([0.07944774, 0.10125179, 0.07709941, 0.07037732, 0.06038963,
       0.04444584, 0.09857071, 0.07104206, 0.11798684, 0.09044124,
       0.18894743]), array([0.09629086, 0.11851939, 0.08530881, 0.06034198, 0.06408189,
       0.03533417, 0.09550056, 0.08161309, 0.09832819, 0.09717294,
       0.16750811]), array([0.08031951, 0.09717838, 0.0

In [4]:
#обучение модели случайного леса
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

#получение важности признаков
feature_importances = rf_model.feature_importances_

#сортировка признаков по важности
indices = (-feature_importances).argsort()[:3]  # Выбираем три самых важных признака

print("Важность признаков с использованием случайного леса:",feature_importances)
print("Самые важные признаки с использованием случайного леса:", indices)

#обучение модели логистической регрессии с L1-регуляризацией
l1_model = LogisticRegression(penalty='l1', solver='liblinear')
l1_model.fit(X_train, y_train)

#получение важности признаков
l1_coefs = l1_model.coef_[0]

#выбор признаков с ненулевыми коэффициентами
indices = np.where(l1_coefs != 0)[0]

print("Важность признаков с использованием L1-регуляризации:",l1_coefs)
print("Самые важные признаки с использованием L1-регуляризации:", indices)

Важность признаков с использованием случайного леса: [0.07596089 0.11146418 0.07402351 0.06959905 0.07839166 0.06581194
 0.10006254 0.08896607 0.07744884 0.11775065 0.14052068]
Самые важные признаки с использованием случайного леса: [10  9  1]
Важность признаков с использованием L1-регуляризации: [ 0.19182459  2.82618978  0.          0.          0.         -0.13651075
 -0.02815469  0.          0.          0.         -0.64107533]
Самые важные признаки с использованием L1-регуляризации: [ 0  1  5  6 10]


In [5]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

#прогноз на тестовом наборе
predictions = rf_model.predict(X_test)

#точность работы модели
accuracy = accuracy_score(y_test, predictions)

print("Точность случайного леса на отложенной выборке:", accuracy)

#точность работы модели с кросс-валидацией
scores = cross_val_score(rf_model, X, y, cv=5, scoring='accuracy')

print("Точность случайного леса с кросс-валидацией (средняя):", scores.mean())

Точность случайного леса на отложенной выборке: 0.5979020979020979
Точность случайного леса с кросс-валидацией (средняя): 0.5774572895119896


In [14]:
#подбор параметров для случайного леса
rf_model = RandomForestClassifier()
rf_param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [5, 10, 20]}
rf_grid_search = GridSearchCV(rf_model, rf_param_grid, cv=2)

#подбор параметров для градиентного бустинга
gb_model = GradientBoostingClassifier()
gb_param_grid = {'n_estimators': [50, 100, 200], 'learning_rate': [0.1, 0.01]}
gb_grid_search = GridSearchCV(gb_model, gb_param_grid, cv=2)

#оценка качества и времени обучения случайного леса
start_time = time.time()
rf_grid_search.fit(X_test, y_test)
end_time = time.time()
rf_best_model = rf_grid_search.best_estimator_
rf_accuracy = accuracy_score(y, rf_best_model.predict(X))
rf_training_time = end_time - start_time

#оценка качества и времени обучения градиентного бустинга
start_time = time.time()
gb_grid_search.fit(X_test, y_test)
end_time = time.time()
gb_best_model = gb_grid_search.best_estimator_
gb_training_time = end_time - start_time
gb_accuracy = accuracy_score(y, gb_best_model.predict(X))

print("Точность случайного леса: ", rf_accuracy)
print("Время обучения случайного леса: ", rf_training_time)
print("Точность градиентного бустинга: ", gb_accuracy)
print("Время обучения градиентного бустинга: ", gb_training_time)

Точность случайного леса:  0.6780402449693789
Время обучения случайного леса:  1.8183484077453613
Точность градиентного бустинга:  0.6377952755905512
Время обучения градиентного бустинга:  6.7086217403411865
